In this notebook, we will build a 3D map of a scene from a small set of images and then localize an image downloaded from the Internet. This demo was contributed by [Philipp Lindenberger](https://github.com/Phil26AT/).

In [ ]:
!pip install pycolmap==3.10.0
!git clone --quiet --recursive https://github.com/cjdwd/Hierarchical-Localization/
%cd Hierarchical-Localization
!pip install --progress-bar off --quiet -e .
!pip install --progress-bar off --quiet --upgrade plotly
!pip uninstall pycolamp
!pip install pycolmap==3.10.0
!pip install kornia

In [ ]:
import os
# os.chdir('Hierarchical-Localization/')
!pwd

In [ ]:
!git clone --quiet --recursive https://github.com/ikotecha/18794-Project

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from DeepUWSFM import (
    feature_extractor, dense_matcher, feature_matcher, pairing
)

In [ ]:
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path

# from hloc.visualization import plot_images, read_image
from DeepUWSFM.utils import viz, viz_3d
import pycolmap
print(pycolmap.__version__)

3.10.0


In [19]:
images = Path("images")

outputs = Path("outputs_new")
sfm_pairs = outputs / "pairs-netvlad.txt"
sfm_dir = outputs / "sfm_superpoint+superglue"
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

# retrieval_conf = extract_features.confs["netvlad"]
feature_conf = feature_extractor.configs["superpoint"]
dense_conf = dense_matcher.confs['loftr']
matcher_conf = feature_matcher.confs["superglue"]

In [ ]:
references = [str(p.relative_to(images)) for p in (images).iterdir()]
print(len(references), "mapping images")
viz.plot_images([read_image(images / r) for r in references], dpi=25)

feature_extractor.extract_features(feature_conf, images, img_list=references, feat_path=features)
pairing.main(sfm_pairs, image_list=references)
feature_matcher.main(matcher_conf, sfm_pairs, features=features, matches=matches);

features_loftr, matches_loftr = dense_matcher.main(dense_conf, sfm_pairs, images, export_dir=outputs)

model = reconstruction.main(sfm_dir, images, sfm_pairs, features_loftr, matches_loftr, image_list=references)

In [ ]:
import pycolmap

def save_points3D_to_ply(reconstruction, output_file):
    points3D = reconstruction.points3D
    with open(output_file, "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points3D)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("end_header\n")

        for point_id, point in points3D.items():
            x, y, z = point.xyz
            r, g, b = point.color
            f.write(f"{x} {y} {z} {r} {g} {b}\n")

    print(f"Point cloud saved to {output_file}")

output_ply_file = images / "points3D.ply"
save_points3D_to_ply(rec, output_ply_file)